In [57]:
import openai
from openai import OpenAI

import base64
import requests

def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

# Usage
api_key_file = 'api_key.txt'
api_key = load_api_key(api_key_file)
# Set up your OpenAI API key
# set api key with openai
openai.api_key = api_key


In [58]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "hard_images/Haruna_Kawaguchi/0066_01.jpg"
prompt = """
What are the facial features of the celeberty in this image. Can you provide the following details in this structured format:
Gender: [Man, Woman]
Race: [Asian, Black, White, Indian, Mixed]
Skin Color: [Light, Medium, Olive, Rosy, Warm Rosy, Warm Tan, Tan, Dark Tan, Dark, Porcelain]
Estimated Age: [age range ex. late 30s]
Face Shape: [Oval, Round, Square, Heart, Oblong, Diamond, Triangle]
Forehead: [Low, Medium, High]
Eyebrows: [Thin, Average, Thick, Arched, Straight]
Eyes:
  - Color: [Brown, Blue, Green, Hazel, Gray]
  - Size: [Small, Medium, Large]
  - Shape: [Round, Almond, Hooded, Downturned, Upturned]
  - Distance: [Close-set, Average, Wide-set]
Nose:
  - Shape: [Straight, Concave, Convex, Button, Hooked, Bulbous]
  - Size: [Small, Medium, Large]
  - Nostril Shape: [Narrow, Average, Wide, Flared]
Cheeks: [Flat, Curved, High, Prominent]
Lips:
  - Size: [Thin, Average, Full]
  - Shape: [Narrow, Average, Wide]
  - Cupid's Bow: [Absent, Subtle, Prominent]
Chin: [Pointed, Round, Square, Dimpled, Double, Cleft]
Jawline: [Rounded, Angular, Square]
Ears: [Small, Medium, Large, Attached Earlobes, Detached Earlobes]
Hair:
  - Color: [Black, Brown, Blonde, Red, Gray, White, Dyed]
  - Texture: [Straight, Wavy, Curly, Coily]
  - Hairline: [Low, Average, High, Receding]
Build: [Slim, Average, Athletic, Heavy, Overweight]
Distinctive Features: [List any notable features such as freckles, dimples, scars, moles, birthmarks, tattoos, piercings, or other unique characteristics]
"""

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9JG8TnypclRdpnp0Ef4HivLOWOmeP', 'object': 'chat.completion', 'created': 1714377285, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Gender: Woman\nRace: Asian\nSkin Color: Light\nEstimated Age: Early 20s\nFace Shape: Oval\nForehead: Medium\nEyebrows: Average, Straight\nEyes:\n  - Color: Brown\n  - Size: Medium\n  - Shape: Almond\n  - Distance: Average\nNose:\n  - Shape: Straight\n  - Size: Small\n  - Nostril Shape: Narrow\nCheeks: Curved\nLips:\n  - Size: Average\n  - Shape: Average\n  - Cupid's Bow: Subtle\nChin: Round\nJawline: Rounded\nEars: Not visible due to her hairs\nHair:\n  - Color: Black\n  - Texture: Straight\n  - Hairline: Average\nBuild: Slim\nDistinctive Features: Bangs/fringe hairstyle\n\nPlease note that some attributes, such as ear size and texture, are not clearly visible, and some assessments may vary with interpretation."}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_toke

In [59]:
facial_features = response.json()["choices"][0]['message']['content']

In [54]:
facial_features

"Gender: Woman\nRace: Asian\nSkin Color: Light\nEstimated Age: early 20s\nFace Shape: Oval\nForehead: Medium\nEyebrows: Average, Straight\nEyes:\n  - Color: Brown\n  - Size: Medium\n  - Shape: Almond\n  - Distance: Average\nNose:\n  - Shape: Straight\n  - Size: Small\n  - Nostril Shape: Narrow\nCheeks: Flat\nLips:\n  - Size: Full\n  - Shape: Average\n  - Cupid's Bow: Subtle\nChin: Round\nJawline: Angular\nEars: Not visible in the image\nHair:\n  - Color: Black\n  - Texture: Straight\n  - Hairline: Average\nBuild: Slim\nDistinctive Features: The person in the image has a clear complexion and a very natural look with no visible tattoos, piercings, or other unique characteristics on display."

In [55]:
settings = ["Looking to left",
            "Has glasses on",
            "Looking forward, front",
            "Picture taken from above",
            "Looking up at camera",
            "Happy",
            "Shocked",
            "Long Hair",
            "Short Hair",
            "Angry"
            ]
image_prompts = []

In [56]:
for setting in settings:

    # User input for facial features and image characteristics
    image_characteristics = "taken with EOS R 300mm f2.8, real photo, 4k resolution, ar 9:16, v 6"
    example = '"American woman on a movie set, light skin, forty two years old, attractive, dark brown hair, taken with EOS R 300mm f2.8, real photo, soft makeup look, handsome, 4k resolution, ar 9:16, v 6, amber eyes, hair parted down middle, thin nose, realistic, casual photo"'
    # Generate the prompt using the user input
    prompt = f"""
    Given the following facial features and image characteristics, please generate a detailed prompt which will be inputted into 
    Dalle for creating an image of this person in the given setting:

    Facial Features:
    {facial_features}

    Image Characteristics:
    {image_characteristics}

    Setting:
    {setting}

    format answer like this example: Limit answer to 70 tokens:
    {example}
    """
    client = OpenAI(api_key=api_key)
    # Use the OpenAI API to generate the image prompt
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=77,
        n=1,
        stop=None,
        temperature=0.7,
        )
    
    image_prompts.append(response.choices[0].message.content.strip())



KeyboardInterrupt: 

In [ ]:
# Extract the generated image prompt from the API response)
print('[', end="")
for p in image_prompts:
    print(p, end=", ")
print(']', end="")

["Asian woman, early 20s, slim build, oval face, black straight hair, brown almond eyes, small straight nose, full lips, looking to the left, soft natural makeup, clear complexion, taken with EOS R 300mm f2.8, real photo, 4k resolution, ar 9:16, v 6.", "Asian woman in her early 20s, oval face, brown almond eyes, small straight nose, full lips, black straight hair, slim build, wearing glasses, clear complexion, soft makeup, taken with EOS R 300mm f2.8, real photo, 4k resolution, ar 9:16, v 6.", "Asian woman, early 20s, oval face, straight black hair, brown almond eyes, full lips, slim build, natural soft makeup, clear complexion, taken with EOS R 300mm f2.8, real photo, 4k resolution, ar 9:16, v 6, front view", "Asian woman in her early 20s, oval face, black straight hair, brown almond-shaped eyes, full lips, slim build, taken from above with EOS R 300mm f2.8, real photo, 4k resolution, ar 9:16, v 6, natural makeup, clear complexion, realistic image.", "Asian woman in her early 20s, ova